In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
import pytorch_lightning as pl
from pathlib import Path

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
DATA_DIR = Path('../data/input_data')
WRITING_DIR = Path('../data/cnn_models/')
SUBDATA_DIR = Path("../data/input_data/subdata")

os.makedirs(SUBDATA_DIR, exist_ok=True)

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

..\data\input_data\sample_submission.csv
..\data\input_data\X_test.npy
..\data\input_data\X_train.npy
..\data\input_data\X_val.npy
..\data\input_data\y_train.npy
..\data\input_data\y_val.npy
..\data\input_data\subdata\X_subtest.npy
..\data\input_data\subdata\X_subval.npy
..\data\input_data\subdata\y_subtest.npy
..\data\input_data\subdata\y_subval.npy


In [24]:
subtest_labels = np.load(SUBDATA_DIR / 'y_subtest.npy').argmax(axis=-1)

In [25]:
from emnist_prediction.metrics import get_classification_report
from sklearn.metrics import classification_report

In [13]:
import torch.nn as nn


class CustomConv2d(nn.Module):

    def __init__(self, n_channels_in, n_channels_out, kernel_size, pad,
                 use_pooling=False, stride=1, drop=None, batch_norm=True,
                 activ=nn.ReLU):
        super(CustomConv2d, self).__init__()
        # We can use here separable convolutions
        layers = [nn.Conv2d(n_channels_in, n_channels_out, kernel_size, stride=stride,
                            padding=pad)]
        if use_pooling:
            layers.append(nn.MaxPool2d(kernel_size=kernel_size))
        if activ:
            layers.append(activ())
        if batch_norm:
            layers.append(nn.BatchNorm2d(n_channels_out))
        if drop is not None:
            layers.append(nn.Dropout2d(drop))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class Flatten(nn.Module):

    def __init__(self, keep_batch_dim=True):
        super(Flatten, self).__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            # -1 means that it will figure out the size
            return x.view(x.shape[0], -1)
        return x.view(-1)


In [14]:
class BaseCNN(pl.LightningModule):

    def __init__(self, class_weights=None, drop=.5):
        super().__init__()
        
        self.class_weights = class_weights
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self(x)
        loss = F.cross_entropy(predictions, y, weight=self.class_weights)
        predicted_labels = torch.argmax(predictions, axis=-1)
        self.log('train_loss', loss, prog_bar=True, sync_dist=True, on_epoch=True)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        predictions = self(x)
        loss = F.cross_entropy(predictions, y)
        predicted_labels = torch.argmax(predictions, axis=-1)
        self.log('val_loss', loss, prog_bar=True, sync_dist=True, on_epoch=True)
        return {'val_loss': loss}
    

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        predictions = self(x)
        predicted_labels = torch.argmax(predictions, axis=-1)
        return predicted_labels

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    

In [27]:
class DeepCNN(BaseCNN):

    def __init__(self, n_input, n_output, **kwargs):
        super(DeepCNN, self).__init__(**kwargs)
        
        self.conv1 = CustomConv2d(n_input, 32, 8, pad='valid')
        self.conv2 = CustomConv2d(32, 64, 8, pad='valid')
        self.conv3 = CustomConv2d(64, 128, 8, pad='valid')
        self.conv4 = CustomConv2d(128, 256, 4, pad='valid')
        self.flatten = Flatten()
        # self.dropout = nn.Dropout(drop)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    

    
class ExperimentalCNN(BaseCNN):

    def __init__(self, n_input, n_output, **kwargs):
        super().__init__(**kwargs)
        
        self.conv1 = CustomConv2d(n_input, 32, 3, pad=1)
        self.conv2 = CustomConv2d(32, 64, 3, pad=1)
        self.flatten = Flatten()
        # self.dropout = nn.Dropout(drop)
        input_flatten = 50176
        self.fc1 = nn.Linear(input_flatten, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x    

In [16]:
def compute_output_size(input_size, kernel_size, padding=0, 
                        stride=1):
    return (input_size - kernel_size + 2 * padding) / stride + 1

compute_output_size(28, 3, padding=1)

28.0

In [6]:
28 * 28 * 64

50176

In [17]:
X_train = np.load(DATA_DIR / 'X_train.npy')

X_mean = X_train.mean()
X_std = X_train.std()

def normalize(X):
    return (X - X_mean) / X_std

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
from emnist_prediction.constants import CLASS_LABELS, IMG_SIZE
from emnist_prediction.datasets import HandwritingsDataset
from emnist_prediction.transforms import image_transform, ImgTransform, Reshape
from torch.utils.data import DataLoader

batch_size = 32
max_epochs = 30
input_size = 784
n_classes=len(CLASS_LABELS)

image_transform = ImgTransform(image_transform, color_channel=True)
reshape_transform = Reshape(shape=(1, *IMG_SIZE))

train_dataset = HandwritingsDataset(X_file_path=DATA_DIR / 'X_train.npy', y_file_path=DATA_DIR / 'y_train.npy',
                                            data_transform=normalize, sample_transform=image_transform)

train_loader = DataLoader(train_dataset, num_workers=4,
                         batch_size=batch_size, shuffle=True)


val_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subval.npy', SUBDATA_DIR / 'y_subval.npy',
                                             data_transform=normalize, sample_transform=reshape_transform)

val_loader = DataLoader(val_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

test_dataset = HandwritingsDataset(SUBDATA_DIR / 'X_subtest.npy', SUBDATA_DIR / 'y_subtest.npy',
                                             data_transform=normalize, sample_transform=reshape_transform)

test_loader = DataLoader(test_dataset, num_workers=4,
                                batch_size=batch_size, shuffle=False)

In [11]:
cnn_checkpoint_dir = WRITING_DIR / 'cnn_experimental'

cnn = ExperimentalCNN(1, len(CLASS_LABELS))

trainer = pl.Trainer(max_epochs=max_epochs, default_root_dir=cnn_checkpoint_dir, fast_dev_run=True)
trainer.fit(cnn, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name    | Type         | Params
-----------------------------------------
0 | conv1   | CustomConv2d | 384   
1 | conv2   | CustomConv2d | 18.6 K
2 | flatten | Flatten      | 0     
3 | fc1     | Linear       | 25.7 M
4 | fc2     | Linear       | 65.7 K
5 | fc3     | Linear       | 3.4 K 
-----------------------------------------
25.8 M    Trainable params
0         Non-trainable params
25.8 M    Total params
103.115   Total estimated model params size (MB)
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packa

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [21]:
os.listdir(cnn_checkpoint_dir / 'best_results')

['epoch=8-val_loss=0.45-train_loss=0.34.ckpt', 'last.ckpt']

In [28]:
cnn = ExperimentalCNN.load_from_checkpoint(cnn_checkpoint_dir / 'best_results/epoch=8-val_loss=0.45-train_loss=0.34.ckpt', 
                               n_input=1, n_output=n_classes)
trainer = pl.Trainer()
predictions = trainer.predict(cnn, test_loader)

print(classification_report(subtest_labels, torch.cat(predictions).numpy()))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.28      0.98      0.43       320
           1       0.00      0.00      0.00       194
           2       0.97      0.99      0.98       505
           3       0.95      0.85      0.89       228
           4       0.98      0.98      0.98       247
           5       0.98      0.99      0.99       459
           6       0.00      0.00      0.00       126
           7       0.96      0.96      0.96       158
           8       0.98      0.98      0.98       597
           9       0.95      0.94      0.95       188
          10       0.95      0.94      0.95       124
          11       0.98      0.95      0.96       254
          12       0.98      0.99      0.99       450
          13       0.98      0.97      0.97       412
          14       0.96      0.99      0.97      1249
          15       0.98      0.98      0.98       417
          16       0.00      0.00      0.00       130
          17       0.00    

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [29]:
clf_report = get_classification_report(subtest_labels, torch.cat(predictions).numpy())

c:\Users\julia\anaconda3\envs\DataCampTutorials\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
clf_report.sort_values(by=['support'])

,precision,recall,f1_score,support
K,0.951220,0.943548,0.947368,124
G,0.000000,0.000000,0.000000,126
Q,0.000000,0.000000,0.000000,130
Z,0.977612,0.970370,0.973978,135
X,0.000000,0.000000,0.000000,138
H,0.955975,0.962025,0.958991,158
J,0.951613,0.941489,0.946524,188
B,0.000000,0.000000,0.000000,194
D,0.946078,0.846491,0.893519,228
V,0.957346,0.870690,0.911964,232


In [32]:
clf_report[clf_report['f1_score'] == 0.0].index

Index(['B', 'G', 'Q', 'R', 'X'], dtype='object')